In [1]:
from usgs import api

# Set the EarthExplorer catalog
node = 'EE'# this indicates earth explorer website

# Set the Hyperion and Landsat 8 dataset
hyperion_dataset = 'EO1_HYP_PUB'
landsat8_dataset = 'LANDSAT_8'

# Set the scene ids
hyperion_scene_id = 'EO1H1820422014302110K2_SG1_01'
landsat8_scene_id = 'LC80290462015135LGN00'

# Submit requests to USGS servers
api.metadata(hyperion_dataset, node, [hyperion_scene_id])
api.metadata(landsat8_dataset, node, [landsat8_scene_id])

usgs.api.metadata(dataset, node, entityids, extended=False, api_key=None)

AttributeError: module 'usgs.api' has no attribute 'metadata'

## Sentinelsat and Copernicus

In [2]:
# https://towardsdatascience.com/satellite-imagery-access-and-analysis-in-python-jupyter-notebooks-387971ece84b

In [6]:
# https://sentinelsat.readthedocs.io/en/stable/

In [3]:
from sentinelsat import SentinelAPI

user = 'COPERNICUS_USERNAME' 
password = 'COPERNICUS_PASSWORD' 
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

In [ ]:
import geopandas as gpd
import folium 

# santa monica bounds file named nReserve
nReserve = gpd.read_file('/Users/sra/Files/brainstation_2023_ds_capstone/brainstation_2023_ds_capstone/01_capstone_data/shapefiles/santa_monica_bounds/sm_bounds.geojson')

# empty base map in Folium centered around the bounding box
m = folium.Map([34.08483,-118.70617], zoom_start=12)
folium.GeoJson(nReserve).add_to(m)
m

In [ ]:
from shapely.geometry import MultiPolygon, Polygon

footprint = None
for i in nReserve['geometry']:
    footprint = i

In [ ]:
products = api.query(footprint,
                     #date = ('20190601', '20190626'),
                     date = ('20220101', '20221231'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,15)
                    )

In [ ]:
products

In [ ]:
products_gdf = api.to_geodataframe(products)
products_gdf_sorted = products_gdf.sort_values(['cloudcoverpercentage'], ascending=[True])
products_gdf_sorted
products_gdf

In [4]:
# api.download('becdf74b-fb47-4010-84f1-2c271a501266')

In [ ]:
import rasterio

R10 = '/Users/sra/Files/brainstation_2023_ds_capstone/brainstation_2023_ds_capstone/01_capstone_data/notebooks/S2A_MSIL2A_20221224T183801_N0509_R027_T11SLT_20221224T213852.SAFE/GRANULE/L2A_T11SLT_A039206_20221224T184116/IMG_DATA/R10m'
b4 = rio.open(R10+'/T11SLT_20221224T183801_B04_10m.jp2')
b3 = rio.open(R10+'/T11SLT_20221224T183801_B03_10m.jp2')
b2 = rio.open(R10+'/T11SLT_20221224T183801_B02_10m.jp2')


# Create an RGB image 
with rio.open('RGB.tiff','w',driver='Gtiff', width=b4.width, height=b4.height, 
              count=3,crs=b4.crs,transform=b4.transform, dtype=b4.dtypes[0]) as rgb:
    rgb.write(b2.read(1),1) 
    rgb.write(b3.read(1),2) 
    rgb.write(b4.read(1),3) 
    rgb.close()

## another one

In [ ]:
# define filenames

# raster in
rasin='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_clip.tif'
# CRS:
# Coordinate Reference System (CRS)
# Name				EPSG:26911 - NAD83 / UTM zone 11N
# Units				meters
# Method			Universal Transverse Mercator (UTM)
# Celestial body	Earth
# Reference			Static (relies on a datum which is plate-fixed)


# shapefile in
shpin='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/perimeters_sm/santa_monica_fire_perimeters_le2018_merge_shp.shp'
# I reprojected the shapefile to the same crs as the raster above in QGIS
# Coordinate Reference System (CRS)

# Name
# EPSG:26911 - NAD83 / UTM zone 11N
# Units
# meters
# Method
# Universal Transverse Mercator (UTM)
# Celestial body
# Earth
# Reference
# Static (relies on a datum which is plate-fixed)


# raster out
rasout='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_sm_fire.tif'

#                 out     in
result=gdal.Warp(rasout,rasin,cutlineDSName=shpin)

iface.addRasterLayer(rasout)

## https://opensourceoptions.com/blog/how-to-clip-a-raster-to-a-polygon-or-an-extent-with-python-extract-by-mask/

In [ ]:
from osgeo import gdal

# fn_in = r"/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_clip.tif"
rast_in='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/m_3411849_se_11_060_20180722.tif'

# fn_clip = '/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_sm_fire.tif'


# fn_poly = r"path/to/polygon.gpkg"

# # gdal.Warp(fn_clip, fn_in, cutlineDSName=fn_poly, cutlineLayer='polygon', cropToCutline=True)

# # from osgeo import gdal
# # fn_in = r"..\..\course-gdal-python\data\input\USGS_one_meter_x64y486_ID_FEMAHQ_2018.tif"
# fn_in=r'/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_clip.tif'

# # fn_poly = r"..\..\course-gdal-python\data\input\polygon.gpkg"
# fn_poly=r'/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/perimeters_sm/santa_monica_fire_perimeters_le2018_merge.geojson'

# # fn_clip = '../data/output/clip_polygon.tif'
# fn_clip='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_clip_fire.tif'


# ortho_fire=gdal.Warp(fn_clip, fn_in, cutlineDSName=fn_poly, cutlineLayer='polygon', cropToCutline=True)
# ortho_fire
# # gdal.Warp(fn_clip2, fn_in, cutlineDSName=fn_poly, cutlineLayer='polygon', cropToCutline=False)
# # where = "ID = '1'"
# # gdal.Warp(fn_clip_where1, fn_in, cutlineDSName=fn_poly, cutlineLayer='polygon', cropToCutline=True, cutlineWhere=where)
# # where = "ID = '2'"
# # gdal.Warp(fn_clip_where2, fn_in, cutlineDSName=fn_poly, cutlineLayer='polygon', cropToCutline=True, cutlineWhere=where)

## rasterio

In [ ]:
# plot the data

show((data,4),cmap='terrain')

In [ ]:
# open raster in read mode
data=rasterio.open(fp)

In [5]:
type(data)

NameError: name 'data' is not defined

In [ ]:
# try to insert geometry into a geodataframe

polygon=r'/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/santa_monica_water_bounds/sm_bounds_water_epsg26911_nad83_utm11n.geojson'

geo=gpd.GeoDataFrame({'geometry':polygon},index=[0],crs=from_epsg(4326))

In [ ]:
# try to re-project into the same coordinate system as the raster data

geo = geo.to_crs(crs=data.crs.data)

In [ ]:
# get coordinates of the geometry in a format that rasterio wants them
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [7]:
coords = getFeatures(geo)
print(coords)

NameError: name 'getFeatures' is not defined

In [8]:
# Now we are ready to clip the raster with the polygon 
# using the coords variable that we just created. 
# Clipping the raster can be done easily with the mask function 
# that we imported in the beginning from rasterio, and specifying clip=True.

out_img, out_transform = mask(raster=data, shapes=coords, crop=True)

NameError: name 'mask' is not defined

## clip raster to polygon

In [ ]:
import shapely

In [ ]:
shapely.geos.geos_version

In [ ]:
# first, merge perimeters
# https://stackoverflow.com/questions/61035170/merging-polygon-shapefiles-in-python

# this was done in QGIS for now. View merged perimeters, 
# to confirm only one polygon in the dataset
sm_perim_merge=gpd.read_file('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/perimeters_sm/santa_monica_fire_perimeters_le2018_merge.geojson')
sm_perim_merge

In [ ]:
# get coordinates from geometry of sm_perim_merge

sm_perim_merge['geometry']

In [ ]:
import gdal

gdal.UseExceptions()

In [ ]:
import cv2
from PIL import Image

# filepaths
target_tiff_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/img_patch_fire2.tif'
output_location_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches'

# load the image using Pillow
with Image.open('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/img_patch_fire2.tif') as img:
    # convert the image to a format that OpenCV can handle (e.g., JPEG)
    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

# display the image using OpenCV
cv2.imshow('image', img_cv)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Set the directory containing the images
directory = "/Users/sra/temp"

# Set the desired size of the square image
desired_size = 128

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        # Load the image
        image_path = os.path.join(directory, filename)
        image = Image.open(image_path)
        
        # Get the size of the image
        width, height = image.size
        
        # Check if the image is already square
        if width == height:
            continue
        
        # Calculate the new size of the image
        if width < height:
            new_width = desired_size
            new_height = int(desired_size * height / width)
        else:
            new_height = desired_size
            new_width = int(desired_size * width / height)
        
        # Resize the image
        new_image = image.resize((new_width, new_height))
        
        # Create a new blank square image
        new_square_image = Image.new("RGB", (desired_size, desired_size), (255, 255, 255))
        
        # Paste the resized image onto the center of the square image
        left = (desired_size - new_width) // 2
        top = (desired_size - new_height) // 2
        new_square_image.paste(new_image, (left, top))
        
        # Save the new square image
        new_image_path = os.path.join(directory, f"{os.path.splitext(filename)[0]}_square.jpg")
        new_square_image.save(new_image_path)


In [ ]:
import os
import cv2

source_folder = '/Users/sra/temp'
destination_folder = '/Users/sra/temp2'

for filename in os.listdir(source_folder):
    if filename.endswith('.jpg'):
        img_path = os.path.join(source_folder, filename)
        img = cv2.imread(img_path)
        h, w, _ = img.shape
        if h != w:
            # stretch the image to make it square
            img = cv2.resize(img, (128, 128))
        cv2.imwrite(os.path.join(destination_folder, filename), img)


In [ ]:
# # define input and output directories
# input_dir = "input_folder/"
# output_dir = "output_folder/"

# # loop through all files in input directory
# for filename in os.listdir(input_dir):
#     # check if the file is a TIFF image
#     if filename.endswith(".tif"):
#         # open the image using PIL
#         with Image.open(input_dir + filename) as img:
#             width, height = img.size
#             # check if the image is already square
#             if width == height:
#                 img.save(output_dir + filename)
#             else:
#                 # stretch the image to be square (128x128 pixels)
#                 if width > height:
#                     new_height = 128
#                     new_width = int(width * new_height / height)
#                     img = img.resize((new_width, new_height))
#                     left = (new_width - 128) / 2
#                     top = 0
#                     right = (new_width + 128) / 2
#                     bottom = 128
#                     img = img.crop((left, top, right, bottom))
#                     img.save(output_dir + filename)
#                 else:
#                     new_width = 128
#                     new_height = int(height * new_width / width)
#                     img = img.resize((new_width, new_height))
#                     left = 0
#                     top = (new_height - 128) / 2
#                     right = 128
#                     bottom = (new_height + 128) / 2
#                     img = img.crop((left, top, right, bottom))
#                     img.save(output_dir + filename)

In [ ]:
def separateNonSquareImages(input_path, output_path):
    '''
    
    Separates non-square images from square images from a directory
    and sends them to a separate user-defined folder.
    
    Square images are those whose width and height are the same.
    For example, a 128x128-pixel image is square. 29x128-pixel
    image is not square.
    
    ----
    Inputs
    
    >input_path
    directory location of where the images (both square and non-)
    are located
    
    >output_path
    new destination directory for the non-square images
    '''
    
    os.makedirs(output_path, exist_ok=True)
    for filename in os.listdir(input_path):
        image_path = os.path.join(input_path, filename)
        if os.path.isfile(input_path):
            with Image.open(input_path) as im:
                width, height = im.size
                if width != height:
                    new_filepath = os.path.join(output_path, filename)
                    os.rename(input_path, output_path)

In [ ]:
input_path='/Users/sra/temp'
output_path='/Users/sra/temp3'

for filename in os.listdir(input_path):
    print(filename)
    input_path = os.path.join(inputPath, file_name)
            output_path = os.path.join(outputPath, 
                                       file_name.replace(oldExtension,
                                                         newExtension))

    try:
        img=Image.open(filename)
        img_size=img.size
        if img_size[0]!=img_size[1]:
            print('not the same size')
        else:
            print("it's the same size")
    except:
        pass

In [ ]:
import os.path

filename = os.path.join('/Users/sra/temp/patch_nofire_857.jpg')
img = Image.open(filename)
img_size=img.size
if img_size[0]!=img_size[1]:
    print('not the same size')
else:
    print("it's the same size")

In [ ]:
# inputPath_nofire_city='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/patch_nofire_city'
# inputPath_nofire_farm='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/patch_nofire_farm'
# inputPath_fire_fire1='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/patch_fire_fire1'
# inputPath_fire_fire2='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/patch_fire_fire2'

# inputPaths=[inputPath_nofire_city,
#             inputPath_nofire_farm,
#             inputPath_fire_fire1,
#             inputPath_fire_fire2]

In [ ]:
# for inp in (inputPaths):
#     fileDeleter(source=inp)

In [ ]:
def resetNumberedFilenames(source):
    '''
    Resets the numbers within a filename so that 
    they start from zero and increase 
    by 1 for all the files in a directory.
    
    ----
    Inputs:
    
    >source
    the directory where the files are located
    
    ----
    Outputs:
    
    >N/A
    renames files in-place, no further output
    '''
    
    # Sort the files in the directory by name
    files = sorted(os.listdir(source))
    print('1',len(files))
    
    # Use a regular expression to search for the number in each filename
    pattern = re.compile(r'\d+')
    print('2',pattern)
    
    # Loop over each file from the source directory
    for i, filename in enumerate(files):
        
        # Get the file extension
        file_extension = os.path.splitext(filename)[1]
        print('3',file_extension)
        
        # Use the regular expression to find the number in the filename
        number_match = pattern.search(filename)
        print('4',number_match)
        
        if number_match:
            number = int(number_match.group())
            print('5',number)
            
        else:
            number = i
            print('6,else',number)
        
        # Create the new filename with the desired format
        new_filename = f'{os.path.splitext(filename)[0].split("_")[0]}_{i}{file_extension}'
        print('7',new_filename)
        
        # Rename the file
        os.rename(os.path.join(source, filename), os.path.join(source, new_filename))
        print('8, renamed')

## Clip raster to polygon

In [ ]:
from osgeo import gdal, ogr

# Define the input raster and polygon mask
input_raster = "/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/m_3411849_se_11_060_20180722.tif"
mask = "/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/perimeters_sm/santa_monica_fire_perimeters_fire_valid.shp"

# Open the input raster and polygon mask
raster_ds = gdal.Open(input_raster)
mask_ds = ogr.Open(mask)

# Get the mask layer
mask_lyr = mask_ds.GetLayer()

# Get the extent of the mask layer
mask_extent = mask_lyr.GetExtent()

# Set the output file name and format
output_file = "path/to/clipped_raster.tif"
output_format = "GTiff"

# Set the output file resolution
output_res = raster_ds.GetGeoTransform()[1]

# Define the output file size
output_width = int((mask_extent[1] - mask_extent[0]) / output_res)
output_height = int((mask_extent[3] - mask_extent[2]) / output_res)

# Define the warp options
warp_options = gdal.WarpOptions(cutlineDSName=mask, cropToCutline=True, dstSRS=raster_ds.GetProjection(), outputBounds=mask_extent, xRes=output_res, yRes=output_res, width=output_width, height=output_height)

# Call the gdal.Warp() function to clip the raster
clipped_raster_ds = gdal.Warp(output_file, raster_ds, options=warp_options)

# Save clipped raster to a shapefile
output_shp = "path/to/clipped_raster.shp"
gdal.VectorTranslate(output_shp, clipped_raster_ds, format="ESRI Shapefile")

# Save clipped raster to a GeoJSON
output_geojson = "path/to/clipped_raster.geojson"
gdal.VectorTranslate(output_geojson, clipped_raster_ds, format="GeoJSON")

# Clean up
raster_ds = None
mask_ds = None
clipped_raster_ds = None

In [ ]:
from osgeo import gdal, ogr

# Define the input raster and polygon mask
input_raster = "/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/m_3411849_se_11_060_20180722.tif"
mask = "/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/perimeters_sm/santa_monica_fire_perimeters_fire_valid.shp"

# Open the input raster and polygon mask
raster_ds = gdal.Open(input_raster)
mask_ds = ogr.Open(mask)

# Get the mask layer
mask_lyr = mask_ds.GetLayer()

# Get the extent of the mask layer
mask_extent = mask_lyr.GetExtent()

# Set the output file name and format for GeoTIFF
output_file_tif = "path/to/clipped_raster.tif"
output_format_tif = "GTiff"

# Set the output file name and format for GeoJSON
output_file_geojson = "path/to/clipped_raster.geojson"
output_format_geojson = "GeoJSON"

# Set the output file resolution
output_res = raster_ds.GetGeoTransform()[1]

# Define the output file size
output_width = int((mask_extent[1] - mask_extent[0]) / output_res)
output_height = int((mask_extent[3] - mask_extent[2]) / output_res)

# Define the warp options
warp_options = gdal.WarpOptions(cutlineDSName=mask, cropToCutline=True, dstSRS=raster_ds.GetProjection(), outputBounds=mask_extent, xRes=output_res, yRes=output_res, width=output_width, height=output_height)

# Call the gdal.Warp() function to clip the raster
clipped_raster_ds = gdal.Warp('', raster_ds, options=warp_options)

# Save clipped raster to GeoTIFF
output_tif = "path/to/clipped_raster.tif"
gdal.Translate(output_tif, clipped_raster_ds, format=output_format_tif)

# Save clipped raster to GeoJSON
output_geojson = "path/to/clipped_raster.geojson"
gdal.Translate(output_file_geojson, clipped_raster_ds, format=output_format_geojson)

# Save clipped raster to a shapefile
output_shp = "path/to/clipped_raster.shp"
gdal.VectorTranslate(output_shp, clipped_raster_ds, format="ESRI Shapefile")

# Clean up
raster_ds = None
mask_ds = None
clipped_raster_ds = None

## [Extrating Patches from Large Images ~~and Masks~~ for Semantic Segmentation](https://www.youtube.com/watch?v=7IL7LKSLb9I)

Following this tutorial to convert my large fire/nofire images into patches for neural network analysis. The code block below is from this video, with some alterations to adapt it to my use case.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from patchify import patchify
import tifffile as tiff

# large_image_stack_fire=tiff.imread('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_sm_fire.tif')

large_image_stack_patch_fire=tiff.imread('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/img_patch_fire2.tif')

In [ ]:
# updated 
# https://stackoverflow.com/questions/68224588/problem-when-using-patchify-library-to-create-patches

import cv2

# filepaths
target_tiff_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/img_patch_fire2.tif'
output_location_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches'

# read large_image_stack_test
img = cv2.imread(target_tiff_fire)

# cv2.imshow('image',img)

In [ ]:
patches_img = patchify(img, (128,128,3), step=128)

for i in range(patches_img.shape[0]):
    for j in range(patches_img.shape[1]):
        single_patch_img = patches_img[i, j, 0, :, :, :]
        if not cv2.imwrite(output_location_fire + 'image_' + '_'+ str(i)+str(j)+'.jpg', single_patch_img):
            raise Exception("Could not write the image")

In [ ]:
# # city
# directory_='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/_patch_jpg/city'
# prefix_='patch_nofire_city_'
# start_=0

# destination_folder='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/_patch_jpg/city/flow_control_resetFileNumbers0to19835'
# destination='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/_patch_jpg/city'

# if not os.path.exists(destination_folder):
#     for dr,prf in zip(directory_,prefix_):
#         resetFileNumbers(directory=dr,prefix=prf,start=test_start)
        
#     # create the full path to the new folder
#     new_folder_path = os.path.join(destination, 'flow_control_resetFileNumbers0to19835')

#     # create the new folder if it doesn't already exist
#     if not os.path.exists(new_folder_path):
#         os.makedirs(new_folder_path)

#### Rename files and change to proper format

Convert filenames from `patch_fire.X.tif` to `patch_fire_X.tif`, where X is a number with one or more digits.

In [148]:
def fileRenamer(source:str, prefix:str,extension='.tif'):
    '''
    Renames files to the format provided by the user.
    It can help clean an image format to one that can be
    read by modules like Tensorflow.
    
    ----
    Inputs:
    
    >source
    the directory where the files are located
    
    >prefix
    the base part of the filename that will remain
    
    >extension
    defaults to '.tif', but this will ensure you only rename
    certain files that have the specified extension
    
    ----
    Outputs:
    
    >N/A
    renames files in-place, no further output
    '''

    # loop over each file from the source directory
    for filename in os.listdir(source):
        
        # check if the file is the provided `ext` (extension)
        if filename.endswith(extension):
            
            # split the filename into base and extension
            base, ext = os.path.splitext(filename)
            
            # split the base into the prefix and number parts
            prefix, number = base.split('.', 1)
            
            # create the new filename with the desired format
            new_filename = f'{prefix}_{number}{ext}'
            
            # rename the file
            os.rename(os.path.join(source, filename), 
                      os.path.join(source, new_filename))

In [152]:
# make a for loop to rename all the files

source_train_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/train/fire'
source_train_nofire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/train/nofire'
source_valid_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/validation/fire'
source_valid_nofire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/validation/nofire'
source_test_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/test/fire'
source_test_nofire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/test/nofire'


sources=[source_train_fire,
        source_train_nofire,
        source_valid_fire,
        source_valid_nofire,
        source_test_fire,
        source_test_nofire]
 
# flow control
if filePresenceSumChecker(directory=source_train_fire,extension='.tif')<0:
    for src in sources:    
        fileRenamer(source=src,prefix='patch_fire')

In [156]:
directory = '/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/validation/nofire'
extension = '.jpg'

filePresenceSumChecker(directory=directory,extension=extension)

0

In [157]:
# setup for loop

inputPath_train_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/train/fire'
inputPath_train_nofire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/train/nofire'
inputPath_valid_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/validation/fire'
inputPath_valid_nofire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/validation/nofire'
inputPath_test_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/test/fire'
inputPath_test_nofire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/test/nofire'


outputPath_train_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/train/fire'
outputPath_train_nofire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/train/nofire'
outputPath_valid_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/validation/fire'
outputPath_valid_nofire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/validation/nofire'
outputPath_test_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/test/fire'
outputPath_test_nofire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/test/nofire'


inputPaths=[inputPath_train_fire,
            inputPath_train_nofire,
            inputPath_valid_fire,
            inputPath_valid_nofire,
            inputPath_test_fire,
            inputPath_test_nofire]

outputPaths=[outputPath_train_fire,
            outputPath_train_nofire,
            outputPath_valid_fire,
            outputPath_valid_nofire,
            outputPath_test_fire,
            outputPath_test_nofire]

In [158]:
# flow control
if filePresenceSumChecker(directory=outputPath_train_fire,extension='.jpg')==0:
    for inp,outp in zip(inputPaths,outputPaths):
        imageConverter(inputPath=inp,outputPath=outp)

In [159]:
filePresenceSumChecker(directory='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/validation/fire',
                      extension='.jpg')

1164

In [45]:
# dir_path='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/nonsquares/validation/nofire'

input_path='/Users/sra/temp'
output_path='/Users/sra/temp3'

if filePresenceSumChecker(directory=output_path,extension='.jpg') != 0:
    separateNonSquareImages(input_path=input_path,output_path=output_path)

In [59]:
# setup for loop

inputPath_train_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/train/fire'
inputPath_train_nofire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/train/nofire'
inputPath_valid_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/validation/fire'
inputPath_valid_nofire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/validation/nofire'
inputPath_test_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/test/fire'
inputPath_test_nofire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/test/nofire'

destPath_train_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/nonsquares/train/fire'
destPath_train_nofire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/nonsquares/train/nofire'
destPath_valid_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/nonsquares/validation/fire'
destPath_valid_nofire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/nonsquares/validation/nofire'
destPath_test_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/nonsquares/test/fire'
destPath_test_nofire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/nonsquares/test/nofire'

inputPaths=[inputPath_train_fire,
            inputPath_train_nofire,
            inputPath_valid_fire,
            inputPath_valid_nofire,
            inputPath_test_fire,
            inputPath_test_nofire]

destPaths=[destPath_train_fire,
          destPath_train_nofire,
          destPath_valid_fire,
          destPath_valid_nofire,
          destPath_test_fire,
          destPath_test_nofire]

In [60]:
checked_location='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/nonsquares/validation/nofire'

if filePresenceSumChecker(directory=checked_location,extension='.jpg') == 0:
    for inp,des in zip(inputPaths,destPaths):
        moveNonSquareImages(source_folder=inp,destination_folder=des)

In [ ]:
import numpy as np

def createTVTS_edit(start, total_img: int, step=1,
              valid_frac=0.15, test_frac=0.15,
              replace=False, verbose=True, debug=False):
    '''
    Creates three lists for a train/validation/test split of 
    numbered files, such as patches previously made from 
    a larger image to be used in convolutional neural network 
    workflows.
    
    The training fraction of the output is the remainder of
    the sum of the validation fraction and the testing fraction:
    
    train_frac = 1 - (valid_frac + test_frac)
    
    Default splits are:
        0.7    = 1 - (   .15     +    .15 )
    
    Please ensure that you have a reasonable split amongst these
    three groups.
    
    ----
    Inputs:
    
    > start: starting number for the image patches
    > total_img: serves both as total size of images in the patch set
    > step: defaults to 1, the step size in creating a list of numbers
    > valid_frac: the fraction of the numbers that will be split into the
        validation set. Please make the number between 0 and 1
    > test_frac: the fraction of the numbers that will be split into the
        test set. Please make the number between 0 and 1
    > replace: since this function is splitting the numbers, replace defaults
        to False
    > verbose: runs a line of code to check that the splitting was successful
    > debug: helpful print statements to show you what step function is on.
        defaults to not showing these statements
    
    ----
    Outputs:
    
    > train_valid_test_tuple: a tuple of three lists, containing the train, valid, and
        test list that when combined together are the same size as the total_img value
    '''
    
    # create list with each image's number
    # there are `total_img` images each in the fire and nofire datasets
    file_nums = np.arange(start, start + total_img, step)
    if debug:
        print(f'created initial list of size {total_img}')
        print(file_nums)
        
    # create train fraction
    train_frac = 1 - (valid_frac + test_frac)
    if debug:
        print(f'created train_fraction ({train_frac})')
    
    # create train, valid, and test splits    
    trains = np.random.choice(file_nums, size=int(total_img * train_frac), replace=False)
    if debug:
        print('created train list')
        print(trains)
    
    valids = np.random.choice(np.setdiff1d(file_nums, trains), size=int(total_img * valid_frac), replace=False)
    if debug:
        print('created validation list')
    
    tests = np.random.choice(np.setdiff1d(file_nums, np.concatenate((trains, valids))),
                             size=int(total_img * test_frac), replace=False)
    if debug:
        print('created test list')
    
    if verbose:
        print(f'The size of train ({len(trains)}), validation ({len(valids)}), and tests ({len(tests)}) together is {len(trains)+len(valids)+len(tests)}')
        if debug:
            print('printed size of train, validation, and test')
    
    return train_valid_test_tuple

In [ ]:
# nofire
# run function
# total_sum is defined in a cell above
train_valid_test_tuple=createTVTS(start=9919,total_img=19836,\
                                  step=1,verbose=True,\
                                  debug=True)
# train_valid_test_tuple

# sanity checks
nofire_trains=train_valid_test_tuple[0]
nofire_valids=train_valid_test_tuple[1]
nofire_tests=train_valid_test_tuple[2]

print(len(nofire_trains))
print(len(nofire_valids))
print(len(nofire_tests))

print(set(nofire_trains) & set(nofire_valids) & set(nofire_tests))

In [ ]:
import numpy as np

def createTVTS_edit(start, total_img: int, step=1,
              valid_frac=0.15, test_frac=0.15,
              replace=False, verbose=True, debug=False):
    '''
    Creates three lists for a train/validation/test split of 
    numbered files, such as patches previously made from 
    a larger image to be used in convolutional neural network 
    workflows.
    
    The training fraction of the output is the remainder of
    the sum of the validation fraction and the testing fraction:
    
    train_frac = 1 - (valid_frac + test_frac)
    
    Default splits are:
        0.7    = 1 - (   .15     +    .15 )
    
    Please ensure that you have a reasonable split amongst these
    three groups.
    
    ----
    Inputs:
    
    > start: starting number for the image patches
    > total_img: serves both as total size of images in the patch set
    > step: defaults to 1, the step size in creating a list of numbers
    > valid_frac: the fraction of the numbers that will be split into the
        validation set. Please make the number between 0 and 1
    > test_frac: the fraction of the numbers that will be split into the
        test set. Please make the number between 0 and 1
    > replace: since this function is splitting the numbers, replace defaults
        to False
    > verbose: runs a line of code to check that the splitting was successful
    > debug: helpful print statements to show you what step function is on.
        defaults to not showing these statements
    
    ----
    Outputs:
    
    > train_valid_test_tuple: a tuple of three lists, containing the train, valid, and
        test list that when combined together are the same size as the total_img value
    '''
    
    # create list with each image's number
    # there are `total_img` images each in the fire and nofire datasets
    file_nums = np.arange(start, start + total_img, step)
    if debug:
        print(f'created initial list of size {total_img}')
        print(file_nums)
        
    # create train fraction
    train_frac = 1 - (valid_frac + test_frac)
    if debug:
        print(f'created train_fraction ({train_frac})')
    
    # create train, valid, and test splits    
    trains = np.random.choice(file_nums, size=int(total_img * train_frac), replace=False)
    if debug:
        print('created train list')
        print(trains)
    
    valids = np.random.choice(np.setdiff1d(file_nums, trains), size=int(total_img * valid_frac), replace=False)
    if debug:
        print('created validation list')
    
    tests = np.random.choice(np.setdiff1d(file_nums, np.concatenate((trains, valids))),
                             size=int(total_img * test_frac), replace=False)
    if debug:
        print('created test list')
    
    if verbose:
        print(f'The size of train ({len(trains)}), validation ({len(valids)}), and tests ({len(tests)}) together is {len(trains)+len(valids)+len(tests)}')
        if debug:
            print('printed size of train, validation, and test')
    train_valid_test_tuple=tuple(trains,valids,tests)
    return train_valid_test_tuple


In [ ]:
# nofire
# run function
# total_sum is defined in a cell above
train_valid_test_tuple=createTVTS_edit(start=9919,total_img=19836,\
                                  step=1,verbose=True,\
                                  debug=True)
# train_valid_test_tuple

# sanity checks
nofire_trains=train_valid_test_tuple[0]
nofire_valids=train_valid_test_tuple[1]
nofire_tests=train_valid_test_tuple[2]

print(len(nofire_trains))
print(len(nofire_valids))
print(len(nofire_tests))

print(set(nofire_trains) & set(nofire_valids) & set(nofire_tests))

In [ ]:
def copy_files_by_index(source_dir, destination_dir, numbers_list):
    # Get a list of all files in the source directory
    files = os.listdir(source_dir)
    # Initialize a counter to keep track of the file index
    index = 0
    # Loop through the files and copy the ones with the specified indices
    for i in range(len(files)):
        # Check if the current file is in the list of indices
        if index in numbers_list:
            # Construct the source and destination paths for the file
            source_path = os.path.join(source_dir, files[i])
            destination_path = os.path.join(destination_dir, files[i])
            # Copy the file to the destination directory
            shutil.copy(source_path, destination_path)
        # Increment the index counter
        index += 1


In [ ]:
def copy_files_by_index(source_dir, dest_dir, index_list):
    file_list = os.listdir(source_dir)
    for index in index_list:
        if index < len(file_list):
            filename = file_list[index]
            src_path = os.path.join(source_dir, filename)
            dst_path = os.path.join(dest_dir, filename)
            shutil.copyfile(src_path, dst_path)
        else:
            print(f"Index {index} is out of range for the source directory.")